In [1]:
from datetime import datetime
import torch
from torchcam.methods import XGradCAM
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os

In [2]:
NORMALIZATION_MEAN = [0.485, 0.456, 0.406]
NORMALIZATION_STD = [0.229, 0.224, 0.225]

In [3]:
FINDINGS = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule',
    'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema',
    'Fibrosis', 'Pleural_Thickening', 'Hernia'
]


In [4]:
# Suponha que model seja o seu modelo carregado
model = torch.load('results/densenet_best.pth')['model']
model.eval()
model.cpu()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
# Função para carregar a imagem e transformá-la em um tensor
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=NORMALIZATION_MEAN, std=NORMALIZATION_STD)
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

In [6]:
# Função para obter as principais probabilidades e índices
def get_top_probabilities(output, topk=3):
    # Aplicar softmax para obter probabilidades
    probabilities = torch.nn.functional.softmax(output, dim=1)
    top_probs, top_idxs = torch.topk(probabilities, topk)
    return top_probs.squeeze(), top_idxs.squeeze()

In [7]:
def generate_cam(image_tensor, model, top_idx, original_image_size):
    # Inicialize o extrator de CAM com o modelo e a camada-alvo
    cam_extractor = XGradCAM(model,
                             target_layer=model.features.norm5)  # Verifique se 'norm5' é o correto para o seu modelo

    # Realize a passagem para frente através do modelo
    out = model(image_tensor)
    # Calcule o CAM para a classe de interesse
    activation_map = cam_extractor(out.squeeze().argmax().item(), out)

    # Verifica se o mapa de ativação é uma lista e pega o primeiro item se for
    if isinstance(activation_map, list):
        activation_map = activation_map[0]

    # Redimensiona o CAM para o tamanho da imagem de entrada
    result = transforms.Resize(original_image_size)(activation_map.unsqueeze(0))

    # Normaliza o CAM para ter valores entre 0 e 1 para melhor visualização
    result = result.squeeze().numpy()
    result = (result - result.min()) / (result.max() - result.min())

    # Inverter o mapa de ativação para garantir que valores mais altos correspondam a cores mais quentes
    result = 1 - result

    return result

In [8]:
activations = []


def hook_fn(module, input, output):
    activations.append(output)

In [10]:
def visualize_activations(activations):
    # Suponha que queremos visualizar os primeiros 64 filtros se houver muitos
    num_activations = activations.shape[1]
    num_activations = min(num_activations, 64)

    # Definir o tamanho da grade para visualização
    grid_size = int(np.ceil(np.sqrt(num_activations)))

    fig, axes = plt.subplots(grid_size, grid_size, figsize=(20, 20))
    for i in range(grid_size ** 2):
        ax = axes[i // grid_size, i % grid_size]
        if i < num_activations:
            ax.imshow(activations[0, i].detach().numpy(), cmap='viridis')
        ax.axis('off')
    plt.show()

In [11]:
# Função para criar um vídeo das ativações
def create_activation_video(activations_list, video_path):
    with imageio.get_writer(video_path, fps=1) as video:
        for activation in activations_list:
            # A ativação é uma tensor 4D: (1, num_filtros, H, W)
            for filter_index in range(activation.size(1)):
                fig, ax = plt.subplots()
                ax.imshow(activation[0, filter_index].cpu().detach().numpy(), cmap='viridis')
                ax.axis('off')
                # Converta a figura em um array de numpy e adicione como um frame do vídeo
                fig.canvas.draw()
                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                video.append_data(image)
                plt.close(fig)

In [12]:
def save_activation_images(activations, directory):
    for layer_idx, act in enumerate(activations):
        # Normalizar a ativação para visualização
        act = act.detach().squeeze(0)
        if act.dim() == 3:  # Se for uma camada convolucional com múltiplos canais
            act = torch.mean(act, 0)
        act = act - act.min()
        act = act / act.max()

        plt.imshow(act.numpy(), cmap='viridis')
        plt.axis('off')
        plt.savefig(os.path.join(directory, f'layer_{layer_idx}.png'), bbox_inches='tight')
        plt.close()

In [13]:
# Função para criar um GIF a partir das imagens salvas
def create_gif(image_folder, gif_path):
    images = []
    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith('.png'):
            images.append(imageio.imread(os.path.join(image_folder, filename)))
    imageio.mimsave(gif_path, images, duration=0.5)  # Ajuste a duração conforme necessário

In [14]:
# Função modificada para criar o CAM de cada camada
def generate_layer_cams(image_tensor, model):
    layer_cams = []
    # Obter todos os módulos em um dicionário
    for name, module in dict(model.named_modules()).items():
        if isinstance(module, torch.nn.modules.conv.Conv2d):
            # Cria o extrator de CAM para a camada atual
            cam_extractor = XGradCAM(model, target_layer=module)
            # Obter a saída do modelo
            model_output = model(image_tensor)
            # Gera o CAM para a classe com a maior saída
            target_class = model_output.argmax(dim=1).item()
            cam = cam_extractor(target_class, model_output)
            # Verifica se o mapa de ativação é uma lista e pega o primeiro item se for
            if isinstance(cam, list):
                cam = cam[0]
            cam = cam.cpu().detach().numpy()
            # Normaliza o CAM
            cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
            layer_cams.append(cam)
    return layer_cams

In [15]:
from torchvision.transforms.functional import to_pil_image


# Função modificada para processar a imagem e criar um GIF dos CAMs
def process_image(image_path, model):
    image_tensor = load_image(image_path)
    original_image = Image.open(image_path).convert('RGB')
    original_image_size = original_image.size

    # Gerar CAM para cada camada convolucional
    layer_cams = generate_layer_cams(image_tensor, model)

    # Defina o diretório temporário para salvar as imagens de CAM
    cam_dir = 'cams'
    if not os.path.exists(cam_dir):
        os.makedirs(cam_dir)

    # Salvar as imagens de CAM
    for i, cam in enumerate(layer_cams):
        plt.imshow(original_image, alpha=1)
        # Convert the CAM to a PIL image, resize it, and then convert back to a numpy array
        cam_resized = to_pil_image(torch.from_numpy(cam)).resize(original_image_size, Image.LINEAR)
        cam_resized = np.asarray(cam_resized)
        # Now the cam_resized is a 2D array, which can be used in imshow
        plt.imshow(cam_resized, alpha=0.5, cmap='hot')
        plt.axis('off')
        plt.savefig(os.path.join(cam_dir, f'cam_{i:03d}.png'), bbox_inches='tight')
        plt.close()

    # Criar um GIF dos CAMs
    gif_path = 'cam_animation.gif'
    create_gif(cam_dir, gif_path)

    # Opcional: Limpar as imagens temporárias se desejado
    # for file_name in os.listdir(cam_dir):
    #     os.remove(os.path.join(cam_dir, file_name))

    print(f"GIF dos CAMs criado: {gif_path}")

In [16]:
# Exemplo de uso
image_path = 'images/00000013_040.png'
process_image(image_path, model)

C:\Users\Murilo\AppData\Local\Temp\ipykernel_2184\2002942400.py:19: RuntimeWarning: invalid value encountered in divide
  cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
C:\Users\Murilo\AppData\Local\Temp\ipykernel_2184\3419788078.py:22: DeprecationWarning: LINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  cam_resized = to_pil_image(torch.from_numpy(cam)).resize(original_image_size, Image.LINEAR)
C:\Users\Murilo\AppData\Local\Temp\ipykernel_2184\2275048128.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(os.path.join(image_folder, filename)))


GIF dos CAMs criado: cam_animation.gif
